# Rain Prediction in Australia


In [1]:
# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
# Import the required libraries
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix, accuracy_score
import sklearn.metrics as metrics

Importing the Dataset


In [3]:
from pyodide.http import pyfetch

async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())

In [4]:
path='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillUp/labs/ML-FinalAssignment/Weather_Data.csv'

In [5]:
await download(path, "Weather_Data.csv")
filename ="Weather_Data.csv"

In [6]:
df = pd.read_csv("Weather_Data.csv")

In [7]:
df.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,...,92,84,1017.6,1017.4,8,8,20.7,20.9,Yes,Yes
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,W,41,W,E,...,83,73,1017.9,1016.4,7,7,22.4,24.8,Yes,Yes
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,...,88,86,1016.7,1015.6,7,8,23.5,23.0,Yes,Yes
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,...,83,90,1014.2,1011.8,8,8,21.4,20.9,Yes,Yes
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,...,88,74,1008.3,1004.8,8,8,22.5,25.5,Yes,Yes


### Data Preprocessing


One hot encoding to convert categorical variables to binary variables.

In [8]:
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

Replace the values of the 'RainTomorrow' column changing them from a categorical column to a binary column. (Not using the `get_dummies` method because we would end up with two columns for 'RainTomorrow' and we do not want, since 'RainTomorrow' is our target.)


In [9]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)

In [10]:
df_sydney_processed

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,41,17,20,92,...,False,False,False,False,False,True,False,False,False,False
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,41,9,13,83,...,False,False,False,False,False,False,False,False,False,False
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,41,17,2,88,...,False,False,False,False,False,False,False,False,False,False
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,41,22,20,83,...,False,False,False,False,False,False,False,False,False,False
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,41,11,6,88,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3266,6/21/2017,8.6,19.6,0.0,2.0,7.8,37,22,20,73,...,False,False,False,False,True,False,False,False,False,False
3267,6/22/2017,9.3,19.2,0.0,2.0,9.2,30,20,7,78,...,False,False,False,False,False,False,False,False,False,False
3268,6/23/2017,9.4,17.7,0.0,2.4,2.7,24,15,13,85,...,False,False,False,False,False,False,False,False,False,False
3269,6/24/2017,10.1,19.3,0.0,1.4,9.3,43,17,19,56,...,False,False,False,False,False,False,False,True,False,False


In [11]:
df_sydney_processed.columns

Index(['Date', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'RainTomorrow', 'RainToday_No', 'RainToday_Yes',
       'WindGustDir_E', 'WindGustDir_ENE', 'WindGustDir_ESE', 'WindGustDir_N',
       'WindGustDir_NE', 'WindGustDir_NNE', 'WindGustDir_NNW',
       'WindGustDir_NW', 'WindGustDir_S', 'WindGustDir_SE', 'WindGustDir_SSE',
       'WindGustDir_SSW', 'WindGustDir_SW', 'WindGustDir_W', 'WindGustDir_WNW',
       'WindGustDir_WSW', 'WindDir9am_E', 'WindDir9am_ENE', 'WindDir9am_ESE',
       'WindDir9am_N', 'WindDir9am_NE', 'WindDir9am_NNE', 'WindDir9am_NNW',
       'WindDir9am_NW', 'WindDir9am_S', 'WindDir9am_SE', 'WindDir9am_SSE',
       'WindDir9am_SSW', 'WindDir9am_SW', 'WindDir9am_W', 'WindDir9am_WNW',
       'WindDir9am_WSW', 'WindDir3pm_E', 'WindDir3pm_ENE', 'WindDir3pm_ESE',
       'WindDir3

In [12]:
df_sydney_processed.dtypes

Date               object
MinTemp           float64
MaxTemp           float64
Rainfall          float64
Evaporation       float64
                   ...   
WindDir3pm_SSW       bool
WindDir3pm_SW        bool
WindDir3pm_W         bool
WindDir3pm_WNW       bool
WindDir3pm_WSW       bool
Length: 68, dtype: object

### Training Data and Test Data


Setting 'features' or x values and Y (target variable).

In [13]:
df_sydney_processed.drop('Date',axis=1,inplace=True)

In [14]:
df_sydney_processed = df_sydney_processed.astype(float)

In [15]:
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

### Linear Regression


In [16]:
# Split the data for Linear Regression
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=10)

In [17]:
# Create and train a Linear Regression model
LinearReg = LinearRegression()
LinearReg.fit(x_train, y_train)

LinearRegression()

In [18]:
# Predict on the test data
predictions = LinearReg.predict(x_test)

In [19]:
# Metrics Calculation
LinearRegression_MAE = metrics.mean_absolute_error(y_test, predictions)
LinearRegression_MSE = metrics.mean_squared_error(y_test, predictions)
LinearRegression_R2 = metrics.r2_score(y_test, predictions)

Display MAE, MSE, and R2 in a tabular format

In [20]:
Report = pd.DataFrame({
    'Metric': ['MAE', 'MSE', 'R2'],
    'Linear Regression': [LinearRegression_MAE, LinearRegression_MSE, LinearRegression_R2]
})

In [21]:
Report

,Metric,Linear Regression
0,MAE,0.256319
1,MSE,0.115721
2,R2,0.427130


### KNN


In [22]:
# Create and train a KNN model
KNN = KNeighborsClassifier(n_neighbors=4)
KNN.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=4)

In [23]:
predictions = KNN.predict(x_test)

In [24]:
# Metrics Calculation
KNN_Accuracy_Score = accuracy_score(y_test, predictions)
KNN_JaccardIndex = jaccard_score(y_test, predictions)
KNN_F1_Score = f1_score(y_test, predictions)

In [25]:
print('Accuracy Score of the KNN Model is:', KNN_Accuracy_Score)
print('Jaccard Index of the KNN Model is:', KNN_JaccardIndex)
print('F1 Score of the KNN Model is:', KNN_F1_Score)

Accuracy Score of the KNN Model is: 0.8183206106870229
Jaccard Index of the KNN Model is: 0.4251207729468599
F1 Score of the KNN Model is: 0.5966101694915255


### Decision Tree


In [26]:
# Create and train a Decision Tree model
Tree = DecisionTreeClassifier()
Tree.fit(x_train, y_train)

DecisionTreeClassifier()

In [27]:
# Predict using the Decision Tree model
predictions = Tree.predict(x_test)

In [28]:
# Calculate metrics for Decision Tree model
Tree_Accuracy_Score = accuracy_score(y_test, predictions)
Tree_JaccardIndex = jaccard_score(y_test, predictions)
Tree_F1_Score = f1_score(y_test, predictions)

print('Accuracy Score of the Decision Tree Model is:', Tree_Accuracy_Score)
print('Jaccard Index of the Decision Tree Model is:', Tree_JaccardIndex)
print('F1 Score of the Decision Tree Model is:', Tree_F1_Score)

Accuracy Score of the Decision Tree Model is: 0.7526717557251908
Jaccard Index of the Decision Tree Model is: 0.3840304182509506
F1 Score of the Decision Tree Model is: 0.554945054945055


### Logistic Regression


In [29]:
# Split the data for Logistic Regression
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=1)

In [30]:
# Train Logistic Regression model
LR = LogisticRegression(solver='liblinear').fit(x_train, y_train)

In [31]:
# Predict using Logistic Regression model
predictions = LR.predict(x_test)
predict_proba = LR.predict_proba(x_test)

In [32]:
predict_proba

array([[0.73237141, 0.26762859],
       [0.97552763, 0.02447237],
       [0.53317815, 0.46682185],
       ...,
       [0.97935358, 0.02064642],
       [0.69890177, 0.30109823],
       [0.2138034 , 0.7861966 ]])

In [33]:
# Calculate metrics for Logistic Regression model
LR_Accuracy_Score = accuracy_score(y_test, predictions)
LR_JaccardIndex = jaccard_score(y_test, predictions)
LR_F1_Score = f1_score(y_test, predictions)
LR_Log_Loss = log_loss(y_test, predict_proba)

In [34]:
print('Accuracy Score of the Logistic Regression Model is:', LR_Accuracy_Score)
print('Jaccard Index of the Logistic Regression Model is:', LR_JaccardIndex)
print('F1 Score of the Logistic Regression Model is:', LR_F1_Score)
print('Log Loss of the Logistic Regression Model is:', LR_Log_Loss)

Accuracy Score of the Logistic Regression Model is: 0.8381679389312977
Jaccard Index of the Logistic Regression Model is: 0.511520737327189
F1 Score of the Logistic Regression Model is: 0.676829268292683
Log Loss of the Logistic Regression Model is: 0.38103765010675233


### SVM


In [35]:
# Train SVM model
SVM = svm.SVC().fit(x_train, y_train)

In [36]:
# Predict using SVM model
predictions = SVM.predict(x_test)

In [37]:
# Calculate metrics for SVM model
SVM_Accuracy_Score = accuracy_score(y_test, predictions)
SVM_JaccardIndex = jaccard_score(y_test, predictions)
SVM_F1_Score = f1_score(y_test, predictions)

print('Accuracy Score of the SVM Model is:', SVM_Accuracy_Score)
print('Jaccard Index of the SVM Model is:', SVM_JaccardIndex)
print('F1 Score of the SVM Model is:', SVM_F1_Score)

Accuracy Score of the SVM Model is: 0.7221374045801526
Jaccard Index of the SVM Model is: 0.0
F1 Score of the SVM Model is: 0.0


### Report


In [38]:
# Create a report DataFrame
Report = pd.DataFrame({
    'Model': ['Linear Regression', 'KNN', 'Decision Tree', 'Logistic Regression', 'SVM'],
    'Accuracy': [LinearRegression_MAE, KNN_Accuracy_Score, Tree_Accuracy_Score, LR_Accuracy_Score, SVM_Accuracy_Score],
    'Jaccard Index': [None, KNN_JaccardIndex, Tree_JaccardIndex, LR_JaccardIndex, SVM_JaccardIndex],
    'F1-Score': [None, KNN_F1_Score, Tree_F1_Score, LR_F1_Score, SVM_F1_Score],
    'LogLoss': [None, None, None, LR_Log_Loss, None]
})

In [39]:
Report

,Model,Accuracy,Jaccard Index,F1-Score,LogLoss
0,Linear Regression,0.256319,NaN,NaN,NaN
1,KNN,0.818321,0.425121,0.596610,NaN
2,Decision Tree,0.752672,0.384030,0.554945,NaN
3,Logistic Regression,0.838168,0.511521,0.676829,0.381038
4,SVM,0.722137,0.000000,0.000000,NaN


### Interpretation
- Linear Regression: Has a low accuracy score (0.256), indicating it doesn’t perform well for this classification task.
- KNN: Shows good performance with an accuracy of 0.818, a Jaccard Index of 0.425, and an F1-Score of 0.597.
- Decision Tree: Performs decently with an accuracy of 0.750, a Jaccard Index of 0.393, and an F1-Score of 0.564.
- Logistic Regression: The best performer with an accuracy of 0.838, a Jaccard Index of 0.512, an F1-Score of 0.677, and a LogLoss of 0.381.
- SVM: Has a moderate accuracy of 0.722 but fails in Jaccard Index and F1-Score, both being 0.
  
Thus, logistic regression model seems to be the most effective one for this dataset.